In [10]:
# Imports and global variables
import requests
import json
import pandas
from dateutil import parser

BASE_URL = "http://localhost:8080/engine-rest"

In [14]:
# CLEANUP
# delete all process definitions and running instances
# ! we need to delete all the running instances first

def status_code_successful(status_code: int):
    return str(status_code)[0] == '2'


def delete_all_data(target: str):
    POSSIBLE_TARGETS = ["process-instance", "process-definition", "deployment", "decision-definition"]
    if target not in POSSIBLE_TARGETS:
        raise Exception(str(target) + "not a valid data deletion target")
        
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    #print(json.dumps(get_response.json(), indent=2))

    for elem in get_response.json():
        current_id = elem.get('id')
        del_response = requests.delete(BASE_URL + "/" + target + "/" + str(current_id))
        #print(del_response)
        assert(status_code_successful(del_response.status_code))

    
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    assert(len(get_response.json()) == 0)   

    
for elem in ["process-instance", "process-definition", "deployment", "decision-definition"]:
    delete_all_data(elem)



In [15]:
# deploy ONE example process 
# TODO: extent for multiple

multipart_form_data = { 
    'deployment-name': (None, 'store'),
    'data': ('food_testing_2.bpmn', open('food_testing_2.bpmn', 'r')),
}

response = requests.post(BASE_URL + "/deployment/create", files=multipart_form_data)
assert(status_code_successful(response.status_code))

#print(json.dumps(response.json(), indent=2))
for elem in response.json().get('deployedProcessDefinitions'):
    new_process_id = elem

print(new_process_id)

food_testing_2:1:9749dbdb-4d38-11ec-8b2d-c66b414752ee


In [16]:
# start batch of instances
for x in range(10):
    headers = {'Content-Type': 'application/json'}
    response = requests.post("http://localhost:8080/engine-rest/process-definition/" + str(new_process_id)+ "/start", headers=headers)
    #print(json.dumps(response.json(), indent=2))
    assert(status_code_successful(response.status_code))



In [1]:
# poll history service
# calculate durations of process instances by process definition

proc_def_response = requests.get(BASE_URL + "/history/process-definition/cleanable-process-instance-report")
assert(status_code_successful(proc_def_response.status_code))
#print(json.dumps(proc_def_response.json(), indent=2))

historic_df = pandas.DataFrame(columns=['processDefinitionId', 'processDefinitionKey', 'processDefinitionName', 'Duration'])


for elem in proc_def_response.json():
    processDefinitionId = elem.get('processDefinitionId')
    processDefinitionKey = elem.get('processDefinitionKey')
    processDefinitionName = elem.get('processDefinitionName')
    params = {'processDefinitionId': str(processDefinitionId)}
    proc_inst_response = requests.get(BASE_URL + "/history/process-instance", params=params)
    assert(status_code_successful(proc_inst_response.status_code))
    for elem2 in proc_inst_response.json():
        end_datetime = parser.parse(elem2.get('endTime'))
        start_datetime = parser.parse(elem2.get('startTime'))
        duration = (end_datetime - start_datetime)
        historic_df.loc[len(historic_df)] = [processDefinitionId, processDefinitionKey, processDefinitionName, duration]

print(historic_df)
# Calculate mean of duration of all instances
print("Mean: " + str((historic_df.Duration.mean())))

NameError: name 'requests' is not defined

In [ ]:
# TODO
# next steps:
# create Instance Router
# put api calls into methods
# find a way to check if all instances have been terminated.
# retrieve data and store it in a log file (in preparation for the rl env)